<a href="https://colab.research.google.com/github/RavinduKodithuwakku/suicide-thoughts-detection/blob/dev/model_logit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Logistic Regression

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, recall_score, accuracy_score, f1_score, precision_score
from collections import Counter
from prettytable import PrettyTable
     

In [ ]:
SEED = 4222
EPOCHS = 5

In [ ]:
# Load dataset
suicide_detection_df = pd.read_csv('/content/drive/MyDrive/fyp/dataset/suicide_detection_full_cleaned.csv', header=0)
suicide_detection_df.drop(columns=['text'], axis=1, inplace=True)
suicide_detection_df = suicide_detection_df.rename(columns={"cleaned_text": "text"})
classes = {"suicide": 1, "non-suicide": 0}
suicide_detection_df = suicide_detection_df.replace({"class": classes})
suicide_detection_df.head()

,class,text
0,1,sex wife threaten suicide recently leave wife ...
1,0,weird not affect compliment come know girl fee...
2,0,finally hear bad year swear fuck god annoying
3,1,need help help cry hard
4,1,lost hello adam struggle year afraid past year...


In [ ]:
# Split dataset into train, validation and test sets
train_text, test_text, train_labels, test_labels = train_test_split(suicide_detection_df['text'], suicide_detection_df['class'],
                                                                    random_state=SEED,
                                                                    test_size=0.2,
                                                                    stratify=suicide_detection_df['class'])

In [ ]:
#print(test_labels)


78485     1
231462    1
207570    1
141033    1
34959     1
         ..
148915    0
14105     0
130592    0
209921    0
6643      1
Name: class, Length: 46415, dtype: int64


Import vocab

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load the vocabulary
vocab_filename = '/content/drive/MyDrive/fyp/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

Import Embeddings

In [ ]:
# load embedding as a dict
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
	return embedding

Removing out of vocab words

In [ ]:
# clean each line
def clean_line(line, vocab):
  tokens_clean = []
  try:  
    tokens = line.split()
    # filter out tokens not in vocab
    tokens_clean = [w for w in tokens if w in vocab]
  except  Exception as e:
    #print(line) 
    pass

  return [tokens_clean]

# clean entire dataset
def process_lines(data, vocab):
  #print(data,vocab)
  
  lines = list()
  for i in data:
    line = clean_line(i, vocab)
    # add lines to list
    lines += line
  return lines

Document Vector function

In [16]:
def document_vector(doc, embeddings):
    sentence = list()
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    doc = [word for word in doc if word in embeddings.keys()]
    for i in doc:
        word = embeddings[i]
        sentence.append(word)
    if len(sentence) == 0:
        return np.zeros(embeddings[next(iter(embeddings))].shape)
    return np.mean(sentence, axis=0)

In [ ]:
# function for all the data
def all_documents(df, labels_ori, embeddings):
  vec = list()
  labels = list()
  for i in range(len(df)):
    #print(df[i])
    if len(df[i]) == 0:
      continue
    else:
      vec.append(document_vector(df[i], embeddings))
      labels.append(labels_ori.values[i])
  return vec, labels
     

Word2Vec

In [ ]:
word2vec = load_embedding('/content/drive/MyDrive/fyp/embedding_word2vec.txt')

In [22]:
train_clean = process_lines(train_text, vocab)
#print(train_clean[:10])
test_clean = process_lines(test_text, vocab)
#print(test_clean[:10])
train_vec, train_labels_new = all_documents(train_clean, train_labels,word2vec)
#print(train_vec[:10],train_labels_new[:10])
test_vec, test_labels_new = all_documents(test_clean, test_labels, word2vec)
#print(test_vec[:10],test_labels_new[:10])


In [ ]:
#train_vec = np.array(train_vec)
#train_vec = train_vec.reshape(-1, 1)

In [18]:
lr = LogisticRegression()
lr.fit(train_vec, train_labels_new)

LogisticRegression()

In [19]:
y_train_pred = lr.predict(train_vec)
print('Training set accuracy %s' % accuracy_score(train_labels_new, y_train_pred))
print(classification_report(train_labels_new, y_train_pred))

Training set accuracy 0.5018326074770385
              precision    recall  f1-score   support

           0       0.50      0.99      0.67     92730
           1       0.58      0.01      0.03     92798

    accuracy                           0.50    185528
   macro avg       0.54      0.50      0.35    185528
weighted avg       0.54      0.50      0.35    185528



In [20]:
y_test_pred = lr.predict(test_vec)
print('Test set accuracy %s' % accuracy_score(test_labels_new, y_test_pred))
print(classification_report(test_labels_new, y_test_pred))

Test set accuracy 0.5020481200413935
              precision    recall  f1-score   support

           0       0.50      0.99      0.67     23181
           1       0.60      0.01      0.03     23203

    accuracy                           0.50     46384
   macro avg       0.55      0.50      0.35     46384
weighted avg       0.55      0.50      0.35     46384



In [21]:
word2vec_test_accuracy_score = accuracy_score(test_labels_new, y_test_pred)
word2vec_test_precision_score = precision_score(test_labels_new, y_test_pred)
word2vec_test_recall_score = recall_score(test_labels_new, y_test_pred)
word2vec_test_f1_score = f1_score(test_labels_new, y_test_pred)

Glove

In [26]:
# load glove embedding from file
raw_embedding_glove = load_embedding('/content/drive/MyDrive/fyp/glove.twitter.27B.200d.txt')

In [27]:
train_clean_glove = process_lines(train_text, raw_embedding_glove.keys())
test_clean_glove = process_lines(test_text, raw_embedding_glove.keys())
train_vec_glove, train_labels_glove_new = all_documents(train_clean_glove, train_labels, raw_embedding_glove)
test_vec_glove, test_labels_glove_new = all_documents(test_clean_glove, test_labels, raw_embedding_glove)

In [28]:
lr = LogisticRegression()
lr.fit(train_vec_glove, train_labels_glove_new)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
y_train_pred = lr.predict(train_vec_glove)
print('Training set accuracy %s' % accuracy_score(train_labels_glove_new, y_train_pred))
print(classification_report(train_labels_glove_new, y_train_pred))

Training set accuracy 0.879904048299283
              precision    recall  f1-score   support

           0       0.90      0.85      0.88     92719
           1       0.86      0.91      0.88     92791

    accuracy                           0.88    185510
   macro avg       0.88      0.88      0.88    185510
weighted avg       0.88      0.88      0.88    185510



In [33]:
y_test_pred = lr.predict(test_vec_glove)
print('Training set accuracy %s' % accuracy_score(test_labels_glove_new, y_test_pred))
print(classification_report(test_labels_glove_new, y_test_pred))

Training set accuracy 0.8782691188202065
              precision    recall  f1-score   support

           0       0.90      0.85      0.87     23179
           1       0.86      0.90      0.88     23202

    accuracy                           0.88     46381
   macro avg       0.88      0.88      0.88     46381
weighted avg       0.88      0.88      0.88     46381



In [34]:
glove_test_accuracy_score = accuracy_score(test_labels_glove_new, y_test_pred)
glove_test_precision_score = precision_score(test_labels_glove_new, y_test_pred)
glove_test_recall_score = recall_score(test_labels_glove_new, y_test_pred)
glove_test_f1_score = f1_score(test_labels_glove_new, y_test_pred)

Summary

In [35]:
table = PrettyTable()
table.field_names = ['Model - Logistic Regression', 'Accuracy', 'Precision', 'Recall', 'F1 Score']

table.add_row(['Word2Vec', 
               format(word2vec_test_accuracy_score, '.4f'), 
               format(word2vec_test_precision_score, '.4f'), 
               format(word2vec_test_recall_score, '.4f'), 
               format(word2vec_test_f1_score, '.4f')])

table.add_row(['GloVe', 
               format(glove_test_accuracy_score, '.4f'), 
               format(glove_test_precision_score, '.4f'), 
               format(glove_test_recall_score, '.4f'), 
               format(glove_test_f1_score, '.4f')])
print(table)

+-----------------------------+----------+-----------+--------+----------+
| Model - Logistic Regression | Accuracy | Precision | Recall | F1 Score |
+-----------------------------+----------+-----------+--------+----------+
|           Word2Vec          |  0.5020  |   0.5978  | 0.0140 |  0.0273  |
|            GloVe            |  0.8783  |   0.8594  | 0.9047 |  0.8815  |
+-----------------------------+----------+-----------+--------+----------+
